In [2]:
import numpy as np
import pandas as pd
import os
import datetime
import warnings
import sklearn
from sklearn import metrics
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.metrics import classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import cross_val_score

import seaborn as sns
import matplotlib.pyplot as plt
import pandas_ta as ta
import pandas as pd
import yfinance as yf
import numpy as np
import math
import statistics
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import urllib.request, json
from pandas import json_normalize
import math
# The first thing we want to do is import the Pandas library and set the filepath to our data file
import pandas as pd
import pandas_ta as ta
import warnings
from pandas.tseries.offsets import DateOffset
import warnings
import sklearn

import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import scipy.stats as sp
from itertools import groupby, chain
from collections import OrderedDict

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import quandl
quandl.ApiConfig.api_key = 'soxd-469x3Zp4ib_4uzc'

df_MKPRU = quandl.get("BCHAIN/MKPRU", authtoken="soxd-469x3Zp4ib_4uzc") # price
df_MWNUS = quandl.get("BCHAIN/MWNUS", authtoken="soxd-469x3Zp4ib_4uzc")

def get_data_yfinance(asset, interval='1d', start='2011-01-01'):
    date_column = 'Date'
    list_of_days = ['1d', '5d', '1wk', '1mo', '3mo']
    list_of_minutes = ['1m', '2m', '5m', '15m', '30m', '60m', '90m']
    df = yf.download(asset, start=start, threads= False, interval=interval)
    df = df.reset_index()
    if interval== '1h':
        df = df.rename(columns={'index':'Date'})
    elif interval in list_of_minutes:
        df = df.rename(columns={'Datetime':'Date'})
        #df[date_column] = df[date_column].str[:]    
    # Now that we have loaded our data into the dataframe, we can preview it using the print & .head() function
    df[date_column] = pd.to_datetime(df[date_column])
    df = df.sort_values([date_column], ascending=[True]).reset_index(drop=True).sort_values([date_column], ascending=[False])
    df = df.rename(columns={'Date':'date','Open':'open', 'High':'high', 'Low': 'low','Close':'close', 'Volume':'tradecount'})
    return(df)

# Create a Dataframe with dates
first_date = '2009-01-02'
first_date_obj = datetime.strptime(first_date, '%Y-%m-%d')
today_obj = datetime.today()
difference = today_obj - first_date_obj
total_days = difference.days
list_of_dates = []
for i in range(total_days):
    new_day = first_date_obj + timedelta(days=i)
    new_day = new_day.strftime('%Y-%m-%d')
    list_of_dates.append(new_day)
df_dates = pd.DataFrame(list_of_dates, columns=['date'])
df_dates['date'] = pd.to_datetime(df_dates['date'], format='%Y-%m-%d')

df_close_yf = get_data_yfinance('BTC-USD')
df_close_yf = df_close_yf[['date','close']]
df_MKPRU_quandul = df_MKPRU.reset_index().rename(columns={'Date':'date', 'Value':'close'})
df_close_clean = pd.merge(df_dates,df_MKPRU_quandul, on='date', how='left')
df_close_clean = pd.merge(df_close_clean,df_close_yf, on='date', how='left')
df_close_clean['close'] = df_close_clean[['close_x','close_y']].apply(lambda x: x['close_x'] if(np.all(pd.notnull(x['close_x']))) else x['close_y'], axis=1)
df_close_clean = df_close_clean[['date','close']]
df_MWNUS = df_MWNUS.reset_index().rename(columns={'Value':'MWNUS'})

df_MWNUS_diff = df_MWNUS.copy()
diff_1 = df_MWNUS_diff[['MWNUS']].diff(periods=1).rename(columns={'MWNUS':'df_MWNUS_diff1'})
diff_2 = diff_1[['df_MWNUS_diff1']].diff(periods=1).rename(columns={'df_MWNUS_diff1':'MWNUS_diff2'})
df_MWNUS_diff = pd.concat([df_MWNUS_diff,diff_1], axis=1)
df_MWNUS_diff = pd.concat([df_MWNUS_diff,diff_2], axis=1).dropna()
#df_MWNUS_diff['MWNUS_diff1_sma_10'] = pd.Series.to_frame(ta.sma(df_MWNUS_diff['MWNUS_diff1'], length=10))
#df_MWNUS_diff['MWNUS_diff1_sma_20'] = pd.Series.to_frame(ta.sma(df_MWNUS_diff['MWNUS_diff1'], length=20))
#df_MWNUS_diff['MWNUS_diff2_sma_10'] = pd.Series.to_frame(ta.sma(df_MWNUS_diff['MWNUS_diff2'], length=10))
#df_MWNUS_diff['MWNUS_diff2_sma_20'] = pd.Series.to_frame(ta.sma(df_MWNUS_diff['MWNUS_diff2'], length=20))
df_MWNUS_diff


[*********************100%***********************]  1 of 1 completed


,Date,MWNUS,df_MWNUS_diff1,MWNUS_diff2
2,2009-01-04,2.0,0.0,0.0
3,2009-01-05,2.0,0.0,0.0
4,2009-01-06,2.0,0.0,0.0
5,2009-01-07,2.0,0.0,0.0
6,2009-01-08,2.0,0.0,0.0
...,...,...,...,...
5094,2023-01-11,85800920.0,8388.0,836.0
5095,2023-01-12,85806899.0,5979.0,-2409.0
5096,2023-01-13,85815361.0,8462.0,2483.0
5097,2023-01-14,85821342.0,5981.0,-2481.0


In [3]:

df_MWNUS_final = pd.merge(df_close_clean, df_MWNUS_diff, left_on='date', right_on='Date', how='left')
df_MWNUS_final.to_csv('MWNUS_diff_visualization.csv', encoding='Latin', sep=';', decimal=',', index=False)
df_final = df_MWNUS_final.copy()
list_of_names_df = ['df_MWNUS_diff1']
# obtain ribbons
list_of_sma = []
list_of_ribbon = []
for i in list_of_names_df:
    sma_5 = i + '_sma_5'
    list_of_sma.append(sma_5)
    df_final[sma_5] = pd.Series.to_frame(ta.sma(df_final[i], length=5))
    sma_10 = i + '_sma_10'
    list_of_sma.append(sma_10)
    df_final[sma_10] = pd.Series.to_frame(ta.sma(df_final[i], length=10))
    sma_20 = i + '_sma_20'
    list_of_sma.append(sma_20)
    df_final[sma_20] = pd.Series.to_frame(ta.sma(df_final[i], length=20))
    sma_30 = i + '_sma_30'
    list_of_sma.append(sma_30)
    df_final[sma_30] = pd.Series.to_frame(ta.sma(df_final[i], length=30))
    sma_40 = i + '_sma_40'
    list_of_sma.append(sma_40)
    df_final[sma_40] = pd.Series.to_frame(ta.sma(df_final[i], length=40))
    sma_60 = i + '_sma_60'
    list_of_sma.append(sma_60)
    df_final[sma_60] = pd.Series.to_frame(ta.sma(df_final[i], length=60))
    ribbon_5_10 = 'ribbon_' + i + '_5_10'
    list_of_ribbon.append(ribbon_5_10)
    df_final[ribbon_5_10] = (df_final[sma_5] - df_final[sma_10])/df_final[sma_10]
    ribbon_10_20 = 'ribbon_' + i + '_10_20'
    list_of_ribbon.append(ribbon_10_20)
    df_final[ribbon_10_20] = (df_final[sma_10] - df_final[sma_20])/df_final[sma_20]
    ribbon_20_40 = 'ribbon_' + i + '_20_40'
    list_of_ribbon.append(ribbon_20_40)
    df_final[ribbon_20_40] = (df_final[sma_20] - df_final[sma_40])/df_final[sma_40]
    ribbon_30_60 = 'ribbon_' + i + '_30_60'
    list_of_ribbon.append(ribbon_30_60)
    df_final[ribbon_30_60] = (df_final[sma_30] - df_final[sma_60])/df_final[sma_60]

list_of_date = ['date']
list_of_sma_to_calculate_inc = list_of_date + list_of_sma
list_of_ribbon_to_calculate_inc = list_of_date + list_of_ribbon
df_sma_to_calculate_inc = df_final[list_of_sma_to_calculate_inc].copy()
df_ribbon_to_calculate_inc = df_final[list_of_ribbon_to_calculate_inc].copy()


#df_final.to_csv('datos_hash.csv', encoding='Latin', index=False, sep=';', decimal=',')

# Create previous data
def creation_df_before(df_final, days, date_column='date', list_of_columns_to_delete=['close']):
    df_final_before = df_final.copy()
    df_final_before = df_final_before.drop(columns=list_of_columns_to_delete)
    df_final_before[date_column] = pd.DatetimeIndex(df_final_before.date) + pd.DateOffset(days)
    dictionary_change_columns = {}
    list_columns_df_final_before = []
    for i in df_final_before.columns:
        dictionary_change_columns[i] = 'before_1_day_' + str(i)
        list_columns_df_final_before.append('before_1_day_' + str(i))
    df_final_before = df_final_before.rename(columns=dictionary_change_columns)
    return df_final_before

def incremental_calculation(df, *args, days=1, date_column= 'date', columns_to_drop=['close']):
    df_final_before = creation_df_before(df_final, days)
    df_final_before = df.copy()
    df_final_before = df_final_before.drop(columns=columns_to_drop)
    df_final_before[date_column] = pd.DatetimeIndex(df_final_before.date) + pd.DateOffset(days)
    dictionary_change_columns = {}
    if len(args) == 0:
        list_columns_df_final_before = []
        for i in df_final_before.columns:
            dictionary_change_columns[i] = 'before_' + str(days) + '_days_' + str(i)
            if i != date_column:
                list_columns_df_final_before.append('before_' + str(days) + '_days_' + str(i))
        df_final_before = df_final_before.rename(columns=dictionary_change_columns)
        df_final_before = df_final_before.rename(columns={'before_' + str(days) + '_days_' + 'date': 'date'})
    else:
        list_columns_df_final_before = []
        for n in args:
            list_columns_df_final_before.append(n)
        df_final_before = df_final_before[list_columns_df_final_before]
        for i in df_final_before.columns:
            dictionary_change_columns[i] = 'before_' + str(days) + '_days_' + str(i)
            if i != date_column:
                list_columns_df_final_before.append('before_' + str(days) + '_days_' + str(i))
        df_final_before = df_final_before.rename(columns=dictionary_change_columns)
        df_final_before = df_final_before.rename(columns={'before_' + str(days) + '_days_' + 'date': 'date'})
    return(df_final_before, list_columns_df_final_before)

df_sma_before_1_days,list_columns_df_sma_before_1_days = incremental_calculation(df_sma_to_calculate_inc, days=1, columns_to_drop=[])
#df_sma_before_4_days,list_columns_df_sma_before_4_days = incremental_calculation(df_sma_to_calculate_inc, days=4, columns_to_drop=[])
#df_sma_before_10_days,list_columns_df_sma_before_10_days = incremental_calculation(df_sma_to_calculate_inc, days=10, columns_to_drop=[])
df_ribbon_before_1_days,list_columns_df_ribbon_before_1_days = incremental_calculation(df_ribbon_to_calculate_inc, days=1, columns_to_drop=[])
#df_ribbon_before_4_days,list_columns_df_ribbon_before_4_days = incremental_calculation(df_ribbon_to_calculate_inc, days=4, columns_to_drop=[])
#df_ribbon_before_10_days,list_columns_df_ribbon_before_10_days = incremental_calculation(df_ribbon_to_calculate_inc, days=10, columns_to_drop=[])

df_final = pd.merge(df_final, df_sma_before_1_days, on='date', how='left')
#df_final = pd.merge(df_final, df_sma_before_4_days, on='date', how='left')
#df_final = pd.merge(df_final, df_sma_before_10_days, on='date', how='left')
df_final = pd.merge(df_final, df_ribbon_before_1_days, on='date', how='left')
#df_final = pd.merge(df_final, df_ribbon_before_4_days, on='date', how='left')
#df_final = pd.merge(df_final, df_ribbon_before_10_days, on='date', how='left')

df_final=df_final.dropna()

# Creation of incrementales
def get_incremental_value(df_in, list_of_features, days):
    df = df_in.copy()
    previous = 'before_' + str(days) + '_days_'
    inc = 'inc_' + str(days) + '_days_'
    list_of_incremental_values = []
    for i in list_of_features:
        df[inc + str(i)] = 100 * (df[i] - df[previous + str(i)]) / df[previous + str(i)]
    return df
df_total_sma = get_incremental_value(df_final, list_of_sma, 1)
#df_total_sma = get_incremental_value(df_total_sma, list_of_sma, 4)
#df_total_sma = get_incremental_value(df_total_sma, list_of_sma, 10)
df_total_sma_ribbon = get_incremental_value(df_total_sma, list_of_ribbon, 1)
#df_total_sma_ribbon = get_incremental_value(df_total_sma_ribbon, list_of_ribbon, 4)
#df_total_sma_ribbon = get_incremental_value(df_total_sma_ribbon, list_of_ribbon, 10)

# Delete columns unuseful

list_of_columns_to_delete = []
list_of_list_to_delete = [list_of_sma, list_of_names_df, list_of_sma_to_calculate_inc, list_columns_df_sma_before_1_days, \
                          list_columns_df_ribbon_before_1_days]
for i in list_of_list_to_delete:
    for j in i:
        if j != 'date':
            list_of_columns_to_delete.append(j)

df = df_total_sma_ribbon.drop(columns=list_of_columns_to_delete).dropna()
df

,date,close,Date,MWNUS,MWNUS_diff2,ribbon_df_MWNUS_diff1_5_10,ribbon_df_MWNUS_diff1_10_20,ribbon_df_MWNUS_diff1_20_40,ribbon_df_MWNUS_diff1_30_60,inc_1_days_df_MWNUS_diff1_sma_5,inc_1_days_df_MWNUS_diff1_sma_10,inc_1_days_df_MWNUS_diff1_sma_20,inc_1_days_df_MWNUS_diff1_sma_30,inc_1_days_df_MWNUS_diff1_sma_40,inc_1_days_df_MWNUS_diff1_sma_60,inc_1_days_ribbon_df_MWNUS_diff1_5_10,inc_1_days_ribbon_df_MWNUS_diff1_10_20,inc_1_days_ribbon_df_MWNUS_diff1_20_40,inc_1_days_ribbon_df_MWNUS_diff1_30_60
1062,2011-11-30,3.14000,2011-11-30,13.0,-1.0,1.000000,1.000000,1.000000,1.000000,83.333333,83.333333,83.333333,83.333333,83.333333,83.333333,0.000000,0.000000,0.000000,0.000000
1063,2011-12-01,3.00000,2011-12-01,22.0,4.0,1.000000,1.000000,1.000000,1.000000,81.818182,81.818182,81.818182,81.818182,81.818182,81.818182,0.000000,0.000000,0.000000,0.000000
1064,2011-12-02,3.12999,2011-12-02,23.0,-8.0,1.000000,1.000000,1.000000,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000
1065,2011-12-03,2.99000,2011-12-03,28.0,4.0,1.000000,1.000000,1.000000,1.000000,23.809524,23.809524,23.809524,23.809524,23.809524,23.809524,0.000000,0.000000,0.000000,0.000000
1066,2011-12-04,3.10000,2011-12-04,28.0,-5.0,0.538462,1.000000,1.000000,1.000000,-23.076923,0.000000,0.000000,0.000000,0.000000,0.000000,-46.153846,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5122,2023-01-11,17443.68000,2023-01-11,85800920.0,836.0,0.026898,-0.720067,0.584876,0.481459,1.572472,4.146803,0.610638,0.345815,0.510628,0.037680,-49.176468,-1.302786,0.270087,0.953884
5123,2023-01-12,17934.60000,2023-01-12,85806899.0,-2409.0,0.028153,-0.712373,0.569415,0.481193,1.652846,1.528736,-1.187016,0.071512,-0.213561,0.089439,4.666803,-1.068459,-2.643479,-0.055111
5124,2023-01-13,18859.32000,2023-01-13,85815361.0,2483.0,0.013008,-0.706084,0.564131,0.482431,0.927138,2.436057,0.244125,0.072000,0.582776,-0.011569,-53.794628,-0.882856,-0.927978,0.257267
5125,2023-01-14,19933.36000,2023-01-14,85821342.0,-2481.0,0.048244,-0.710186,0.541346,0.482772,0.308966,-3.062779,-1.690749,0.024581,-0.237509,0.001596,270.864671,0.580946,-4.038905,0.070627


In [5]:
df[['date', 'close', 'Date', 'MWNUS', 'MWNUS_diff2',
       'ribbon_df_MWNUS_diff1_5_10', 'ribbon_df_MWNUS_diff1_10_20',
       'ribbon_df_MWNUS_diff1_20_40', 'ribbon_df_MWNUS_diff1_30_60']]

,date,close,Date,MWNUS,MWNUS_diff2,ribbon_df_MWNUS_diff1_5_10,ribbon_df_MWNUS_diff1_10_20,ribbon_df_MWNUS_diff1_20_40,ribbon_df_MWNUS_diff1_30_60
1062,2011-11-30,3.14000,2011-11-30,13.0,-1.0,1.000000,1.000000,1.000000,1.000000
1063,2011-12-01,3.00000,2011-12-01,22.0,4.0,1.000000,1.000000,1.000000,1.000000
1064,2011-12-02,3.12999,2011-12-02,23.0,-8.0,1.000000,1.000000,1.000000,1.000000
1065,2011-12-03,2.99000,2011-12-03,28.0,4.0,1.000000,1.000000,1.000000,1.000000
1066,2011-12-04,3.10000,2011-12-04,28.0,-5.0,0.538462,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...
5122,2023-01-11,17443.68000,2023-01-11,85800920.0,836.0,0.026898,-0.720067,0.584876,0.481459
5123,2023-01-12,17934.60000,2023-01-12,85806899.0,-2409.0,0.028153,-0.712373,0.569415,0.481193
5124,2023-01-13,18859.32000,2023-01-13,85815361.0,2483.0,0.013008,-0.706084,0.564131,0.482431
5125,2023-01-14,19933.36000,2023-01-14,85821342.0,-2481.0,0.048244,-0.710186,0.541346,0.482772


In [ ]:



def sma_long_and_short_calculation(df_final, indicator, small_sma, big_sma, date_column='date', close_column='close'):
    df_calculo = df_final[[date_column, \
                           'df_'+indicator, \
                           'df_'+ indicator +'_sma_' + str(small_sma), \
                           'df_'+indicator + '_sma_' + str(big_sma), \
                           'before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma), \
                           'ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma), \
                           close_column]]
    # 2  long y short signals calculation
    df_calculo["long_signal"] = df_calculo[['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma),\
                                            'ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)\
                                           ]].apply(lambda x: 1 if x['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma)]<0 and x['ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]>=0  else 0, axis=1)
   
    df_calculo["short_signal"] = df_calculo[['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma),\
                                             'ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]].apply(lambda x: 1 if x['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma)]>0 and x['ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]<=0  else 0, axis=1)
    print(df_calculo)
    # 3 calculation maximum, minimum, differences
    list_of_operations = []
    list_of_entry_price = []
    list_of_exit_price = []
    list_of_date_entry=[]
    list_of_date_exit=[]
    list_of_maximum=[]
    list_of_minimum=[]
    list_of_MWNUS_diff2=[]

    maximum_value=0
    minimum_value=0
    entry_long_value=0
    entry_short_value=0
    MWNUS_diff2=0
   
    date_enter_short = df_calculo.iloc[0][date_column]
    date_enter_long = df_calculo.iloc[0][date_column]
   
    for i in range(len(df_calculo)):
        if df_calculo.iloc[i]['short_signal']== 1:
            if df_calculo.iloc[i][close_column] < minimum_value:
                minimum_value = df_calculo.iloc[i][close_column]
            elif df_calculo.iloc[i][close_column] > maximum_value:
                maximum_value = df_calculo.iloc[i][close_column]
            #creation new values in long list
            entry_short_value = df_calculo.iloc[i][close_column]
            date_enter_short = df_calculo.iloc[i][date_column]
            previous_maximum = maximum_value
            previous_minimum = minimum_value
            minimum_value = df_calculo.iloc[i][close_column]
            maximum_value = df_calculo.iloc[i][close_column]
            previous_MWNUS_diff2=MWNUS_diff2
            MWNUS_diff1 = df_calculo.iloc[i]['df_MWNUS_diff1']
            if entry_short_value>0:
                list_of_operations.append('long')
                list_of_entry_price.append(entry_long_value)
                list_of_exit_price.append(entry_short_value)
                list_of_date_entry.append(date_enter_long)
                list_of_date_exit.append(date_enter_short)
                list_of_maximum.append(previous_maximum)
                list_of_minimum.append(previous_minimum)
                list_of_MWNUS_diff2.append(previous_MWNUS_diff2)

           
        elif df_calculo.iloc[i]['long_signal']== 1:
            if df_calculo.iloc[i][close_column] < minimum_value:
                minimum_value = df_calculo.iloc[i][close_column]
            elif df_calculo.iloc[i][close_column] > maximum_value:
                maximum_value = df_calculo.iloc[i][close_column]
            date_enter_long = df_calculo.iloc[i][date_column]
            entry_long_value = df_calculo.iloc[i][close_column]
            previous_maximum = maximum_value
            previous_minimum = minimum_value
            maximum_value = df_calculo.iloc[i][close_column]
            minimum_value = df_calculo.iloc[i][close_column]
            previous_MWNUS_diff2=MWNUS_diff2
            MWNUS_diff1 = df_calculo.iloc[i]['df_MWNUS_diff1']
            if entry_short_value>0:
                list_of_operations.append('short')
                list_of_entry_price.append(entry_short_value)
                list_of_exit_price.append(entry_long_value)
                list_of_date_entry.append(date_enter_short)
                list_of_date_exit.append(date_enter_long)
                list_of_maximum.append(previous_maximum)
                list_of_minimum.append(previous_minimum)
                list_of_MWNUS_diff2.append(previous_MWNUS_diff2)

       
        if df_calculo.iloc[i][close_column] < minimum_value:
            minimum_value = df_calculo.iloc[i][close_column]
        elif df_calculo.iloc[i][close_column] > maximum_value:
            maximum_value = df_calculo.iloc[i][close_column]
    data = pd.DataFrame(columns = ['indicator', 'operation', 'entry_price', 'exit_price', 'maximum_price', 'minimum_price', 'entry_date' ,'exit_date', 'MWNUS_diff2'])
    df_out = data.append(pd.DataFrame({'indicator':indicator, 'operation':list_of_operations, 'entry_price':list_of_entry_price, 'exit_price':list_of_exit_price, 'maximum_price':list_of_maximum, 'minimum_price':list_of_minimum, 'entry_date':list_of_date_entry ,'exit_date':list_of_date_exit, 'MWNUS_diff2': list_of_MWNUS_diff2}), ignore_index = True)

    return(df_out)

df_out = sma_long_and_short_calculation(df_final, 'MWNUS_diff1', 10, 20)

df_out.to_csv('long_short_signals_MWNUS.csv', encoding='Latin', sep=';', decimal=',', index=False)